In [ ]:
import pandas as pd
import random

# Load your dataset
df = pd.read_csv('/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

# Show column names to debug if needed
print("Columns in the dataset:", df.columns.tolist())

# Try to find the label column (case-insensitive, strip spaces)
label_col = next((col for col in df.columns if 'label' in col.lower()), None)

if not label_col:
    raise ValueError("No label column found in the dataset.")

# Step 1: Select only the first 50 rows
df_50 = df.head(50).copy()

# Step 2: Randomly pick 20 unique indices for attack labels
attack_indices = random.sample(range(50), 20)

# Step 3: Create new label columns
df_50['is_attack'] = 'benign'
df_50.loc[attack_indices, 'is_attack'] = 'attack'

# Step 4: Create binary column from 'is_attack'
df_50['attack_binary'] = df_50['is_attack'].map({'benign': 0, 'attack': 1})

# Step 5: Show final result
print(f"\n✅ Using label column: '{label_col}'")
print(df_50[[label_col, 'is_attack', 'attack_binary']])

# Optional: Save to CSV
df_50.to_csv('/content/small_intrusion_dataset.csv', index=False)


Columns in the dataset: [' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import smtplib
from email.mime.text import MIMEText

# Load the mini dataset
df = pd.read_csv('/content/small_intrusion_dataset.csv')

# Step 1: Prepare features (drop labels)
X = df.drop(columns=['is_attack', 'attack_binary'])

# Drop non-numeric columns (keep it simple for now)
X = X.select_dtypes(include='number')

# Step 2: Prepare target
y = df['attack_binary']

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict on test set
y_pred = model.predict(X_test)

# Step 6: Evaluation
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

# Step 7: Check if any attack was predicted
if 1 in y_pred:
    print("\n⚠️ Intrusion(s) detected! Sending alert...")

    # Email alert setup
    sender_email = "aditya27projects@gmail.com"       # Replace with your email
    receiver_email = "aditya27kankarwal@gmail.com" # Replace with target email
    password = "vopx ykrn nqrr fgxi"             # Use app-specific password (e.g., Gmail)

    subject = "🚨 Intrusion Detected in Network"
    body = "The ML model has detected a potential intrusion in the network.\nPlease check the logs or take action."

    # Compose email
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email

    try:
        # Connect to SMTP server
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender_email, password)
        server.send_message(msg)
        server.quit()
        print("✅ Alert email sent successfully.")
    except Exception as e:
        print("❌ Failed to send email:", e)
else:
    print("\n✅ No intrusions detected. All good.")




📊 Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.86      0.80         7
           1       0.50      0.33      0.40         3

    accuracy                           0.70        10
   macro avg       0.62      0.60      0.60        10
weighted avg       0.68      0.70      0.68        10


⚠️ Intrusion(s) detected! Sending alert...
✅ Alert email sent successfully.


In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# === CONFIG SECTION ===

EMAIL_ALERTS = True   # Set to False if you don't want email alerts
YOUR_EMAIL = "aditya27projects@gmail.com"
YOUR_PASSWORD = "**** **** ****"  # Use App Password for Gmail
TO_EMAIL = "aditya27kankarwal@gmail.com"

# === STEP 1: Load Dataset ===
df = pd.read_csv('/content/small_intrusion_dataset.csv')

# === STEP 2: Intrusion Detection ===
print("🔍 Smart Network Intrusion Notifier (SNIN) - Alert Log\n")

alert_messages = []
for idx, row in df.iterrows():
    if row['attack_binary'] == 1:
        msg = f"🚨 Intrusion detected in Packet #{idx + 1} — Marked as '{row['is_attack'].upper()}'"
        print(msg)
        alert_messages.append(msg)

# === STEP 3: Summary ===
print(f"\n✅ Total Packets Scanned: {len(df)}")
print(f"⚠️ Total Intrusions Detected: {len(alert_messages)}")

# === STEP 4: Send Email Alert (optional) ===
if EMAIL_ALERTS and alert_messages:
    try:
        # Create the email content
        subject = "🚨 SNIN Alert: Intrusions Detected in Network Traffic"
        body = "\n".join(alert_messages)

        message = MIMEMultipart()
        message["From"] = YOUR_EMAIL
        message["To"] = TO_EMAIL
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        # Connect to Gmail SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(YOUR_EMAIL, YOUR_PASSWORD)
        server.send_message(message)
        server.quit()

        print(f"\n📧 Alert email sent to {TO_EMAIL}!")
    except Exception as e:
        print("❌ Failed to send email alert:", str(e))


🔍 Smart Network Intrusion Notifier (SNIN) - Alert Log

🚨 Intrusion detected in Packet #1 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #4 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #6 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #7 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #16 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #18 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #19 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #22 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #24 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #27 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #29 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #30 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #31 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #32 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #37 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #39 — Marked as 'ATTACK'
🚨 Intrusion detected in Packet #41 — Marked as 'ATTAC

In [ ]:
import pandas as pd
import numpy as np
import random

# ========= EGT Setup =========

# Define strategies and payoff matrix for the defender
# (The tuple's first element is the payoff for the defender.)
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

# Initialize defender strategy proportions (starting equally)
defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(defender_strategy, actual, prediction):
    """
    Update defender strategy using replicator dynamics.

    Parameters:
      defender_strategy (dict): current proportions for 'aggressive' and 'passive'
      actual (int): 1 if the packet is an attack, 0 if benign
      prediction (int): 1 if IDS predicted an attack, 0 otherwise

    The payoff for the defender is chosen from the payoff_matrix based on the actual outcome.
    """
    outcome = 'attack' if actual == 1 else 'benign'

    # Retrieve payoffs for each strategy (defender's perspective)
    payoff_aggressive = payoff_matrix[('aggressive', outcome)][0]
    payoff_passive = payoff_matrix[('passive', outcome)][0]

    # Calculate the average payoff
    avg_payoff = (defender_strategy['aggressive'] * payoff_aggressive +
                  defender_strategy['passive'] * payoff_passive)

    # Apply replicator dynamics update
    defender_strategy['aggressive'] += learning_rate * defender_strategy['aggressive'] * (payoff_aggressive - avg_payoff)
    defender_strategy['passive']    += learning_rate * defender_strategy['passive']    * (payoff_passive    - avg_payoff)

    # Normalize to maintain proportions summing to 1
    total = defender_strategy['aggressive'] + defender_strategy['passive']
    defender_strategy['aggressive'] /= total
    defender_strategy['passive'] /= total

    return defender_strategy

# ========= Load and Prepare Data =========

# Load your small dataset with binary labels from the intrusion detection preprocessing step.
df = pd.read_csv('/content/small_intrusion_dataset.csv')

print("🔍 Starting EGT-enhanced Intrusion Detection Simulation\n")

# ========= Intrusion Detection with EGT =========

# Loop over each packet (row) in the dataset, simulate a prediction, and update strategy.
for i, row in df.iterrows():
    # Actual label from dataset: 1 indicates an attack, 0 is benign.
    actual = int(row['attack_binary'])

    # Simulate the model's predicted probability:
    # For attack packets, assume the model is more likely to predict higher probability.
    if actual == 1:
        prediction_prob = random.uniform(0.6, 0.9)
    else:
        prediction_prob = random.uniform(0.1, 0.4)

    # Determine the current dominant strategy based on the defender's evolving strategy.
    # Use 'aggressive' if its proportion is higher, else 'passive'
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'

    # Set detection threshold: aggressive IDS uses a lower threshold (more alerts) and vice versa.
    threshold = 0.3 if strategy == 'aggressive' else 0.7

    # Make prediction: if the probability exceeds the threshold, predict an attack.
    prediction = 1 if prediction_prob > threshold else 0

    # Update the defender strategy based on the actual outcome vs. the prediction.
    defender_strategy = update_strategy(defender_strategy, actual, prediction)

    # Log the decision for this packet.
    print(f"[{i+1:2d}] Strategy: {strategy.upper()} | Actual: {actual} | Predicted: {prediction} | Aggressive Ratio: {defender_strategy['aggressive']:.2f}")

# ========= Final Summary =========

total_attacks = df['attack_binary'].sum()
print(f"\n✅ Total Packets Scanned: {len(df)}")
print(f"⚠️ Total Attacks in dataset: {total_attacks}")


🔍 Starting EGT-enhanced Intrusion Detection Simulation

[ 1] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.45
[ 2] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.40
[ 3] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.35
[ 4] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.31
[ 5] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.26
[ 6] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.23
[ 7] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.19
[ 8] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.16
[ 9] Strategy: PASSIVE | Actual: 1 | Predicted: 0 | Aggressive Ratio: 0.20
[10] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.17
[11] Strategy: PASSIVE | Actual: 1 | Predicted: 0 | Aggressive Ratio: 0.21
[12] Strategy: PASSIVE | Actual: 0 | Predicted: 0 | Aggressive Ratio: 0.18
[13] Strategy: PASSIVE | Actual: 1 | Predict

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import numpy as np
import random
from collections import deque

# === Email Setup ===
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # Use App Password (Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_email(packet_no, strategy, reason, prediction_prob):
    subject = f"🚨 Intrusion Detected at Packet #{packet_no}"
    body = f"""
    Intrusion Alert 🚨

    ➤ Packet Number: {packet_no}
    ➤ Strategy Used: {strategy.upper()} ({reason})
    ➤ Prediction Probability: {prediction_prob:.2f}

    The Smart Network Intrusion Notifier (SNIN) flagged this packet as a potential attack.
    """
    msg = MIMEMultipart()
    msg['From'] = EMAIL_SENDER
    msg['To'] = EMAIL_RECEIVER
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
            print(f"📧 Email sent for packet #{packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# === Load Dataset ===
df = pd.read_csv('/content/small_intrusion_dataset.csv')

# === Define EGT Strategies & Payoff Matrix ===
strategies = ['aggressive', 'passive']
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

# === Initialize Strategy State ===
defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(defender_strategy, actual, prediction):
    outcome = 'attack' if actual == 1 else 'benign'
    payoff_aggressive = payoff_matrix[('aggressive', outcome)][0]
    payoff_passive = payoff_matrix[('passive', outcome)][0]

    avg_payoff = (defender_strategy['aggressive'] * payoff_aggressive +
                  defender_strategy['passive'] * payoff_passive)

    defender_strategy['aggressive'] += learning_rate * defender_strategy['aggressive'] * (payoff_aggressive - avg_payoff)
    defender_strategy['passive'] += learning_rate * defender_strategy['passive'] * (payoff_passive - avg_payoff)

    total = defender_strategy['aggressive'] + defender_strategy['passive']
    defender_strategy['aggressive'] /= total
    defender_strategy['passive'] /= total

    return defender_strategy

# === Short-Term Memory ===
recent_attacks = deque(maxlen=5)

print("🔐 SNIN - Intrusion Detection with Email Alerts\n")

for i, row in df.iterrows():
    actual = int(row['attack_binary'])

    # Simulate prediction probability
    prediction_prob = random.uniform(0.6, 0.95) if actual == 1 else random.uniform(0.1, 0.4)

    recent_attacks.append(actual)

    # Strategy Selection
    if sum(recent_attacks) >= 3:
        strategy = 'aggressive'
        reason = 'forced by memory'
    else:
        strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'
        reason = 'EGT chosen'

    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    # Update EGT strategy
    defender_strategy = update_strategy(defender_strategy, actual, prediction)

    # Print log
    print(f"[{i+1:02d}] Strategy: {strategy.upper():>9} ({reason}) | Actual: {actual} | Predicted: {prediction} | Aggressive %: {defender_strategy['aggressive']:.2f}")

    # Send alert if predicted attack
    if prediction == 1:
        send_email(i + 1, strategy, reason, prediction_prob)


🔐 SNIN - Intrusion Detection with Email Alerts

[01] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.45
[02] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.40
[03] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.35
[04] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.31
[05] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.26
[06] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.23
[07] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.19
[08] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.16
[09] Strategy:   PASSIVE (EGT chosen) | Actual: 1 | Predicted: 1 | Aggressive %: 0.20
📧 Email sent for packet #9
[10] Strategy:   PASSIVE (EGT chosen) | Actual: 0 | Predicted: 0 | Aggressive %: 0.17
[11] Strategy:   PASSIVE (EGT chosen) | Actual: 1 | Predicted: 1 

In [ ]:
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # Use App Password (Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_alert_email(packet_no, prediction_prob, strategy, actual):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    IDS Strategy: {strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[:26]

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    if prediction == 1:
        print(f"[Packet {i+1}] 🚨 Intrusion Detected | Strategy: {strategy.upper()} | Prediction: {prediction_prob:.2f}")
        send_alert_email(i+1, prediction_prob, strategy, actual)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}")


[Packet 1] IDS State → Aggressive: 0.45, Passive: 0.55
[Packet 2] IDS State → Aggressive: 0.40, Passive: 0.60
[Packet 3] IDS State → Aggressive: 0.35, Passive: 0.65
[Packet 4] IDS State → Aggressive: 0.31, Passive: 0.69
[Packet 5] IDS State → Aggressive: 0.26, Passive: 0.74
[Packet 6] IDS State → Aggressive: 0.23, Passive: 0.77
[Packet 7] IDS State → Aggressive: 0.19, Passive: 0.81
[Packet 8] IDS State → Aggressive: 0.16, Passive: 0.84
[Packet 9] IDS State → Aggressive: 0.20, Passive: 0.80
[Packet 10] IDS State → Aggressive: 0.17, Passive: 0.83
[Packet 11] 🚨 Intrusion Detected | Strategy: PASSIVE | Prediction: 0.75
📧 Email alert sent for Packet 11
[Packet 11] IDS State → Aggressive: 0.21, Passive: 0.79
[Packet 12] IDS State → Aggressive: 0.18, Passive: 0.82
[Packet 13] IDS State → Aggressive: 0.22, Passive: 0.78
[Packet 14] IDS State → Aggressive: 0.19, Passive: 0.81
[Packet 15] 🚨 Intrusion Detected | Strategy: PASSIVE | Prediction: 0.82
📧 Email alert sent for Packet 15
[Packet 15] IDS

In [ ]:
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = "aditya27projects@gmail.com"
EMAIL_PASSWORD = "**** **** **** ****"
EMAIL_RECEIVER = "aditya27kankarwal@gmail.com"

def send_alert_email(packet_no, prediction_prob, strategy, actual):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    IDS Strategy: {strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[26:50]

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.6, 0.95)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    if prediction == 1:
        print(f"[Packet {i+26}] 🚨 Intrusion Detected | Strategy: {strategy.upper()} | Prediction: {prediction_prob:.2f}")
        send_alert_email(i+26, prediction_prob, strategy, actual)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+26}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}")


[Packet 52] 🚨 Intrusion Detected | Strategy: PASSIVE | Prediction: 0.86
📧 Email alert sent for Packet 52
[Packet 52] IDS State → Aggressive: 0.57, Passive: 0.42
[Packet 53] 🚨 Intrusion Detected | Strategy: AGGRESSIVE | Prediction: 0.41
📧 Email alert sent for Packet 53
[Packet 53] IDS State → Aggressive: 0.53, Passive: 0.47
[Packet 54] 🚨 Intrusion Detected | Strategy: AGGRESSIVE | Prediction: 0.79
📧 Email alert sent for Packet 54
[Packet 54] IDS State → Aggressive: 0.60, Passive: 0.40
[Packet 55] 🚨 Intrusion Detected | Strategy: AGGRESSIVE | Prediction: 0.67
📧 Email alert sent for Packet 55
[Packet 55] IDS State → Aggressive: 0.67, Passive: 0.33
[Packet 56] IDS State → Aggressive: 0.63, Passive: 0.37
[Packet 57] 🚨 Intrusion Detected | Strategy: AGGRESSIVE | Prediction: 0.41
📧 Email alert sent for Packet 57
[Packet 57] IDS State → Aggressive: 0.58, Passive: 0.42
[Packet 58] IDS State → Aggressive: 0.53, Passive: 0.47
[Packet 59] 🚨 Intrusion Detected | Strategy: AGGRESSIVE | Prediction: 0

In [ ]:
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_alert_email(packet_no, prediction_prob, strategy, actual):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    IDS Strategy: {strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[:26]
df[' Label'] = df[' Label'].str.strip()
df['is_attack'] = df[' Label'].apply(lambda x: 'attack' if x.upper() != 'BENIGN' else 'benign')
df['attack_binary'] = df['is_attack'].apply(lambda x: 1 if x == 'attack' else 0)

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1
consecutive_attacks = 0

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']

    # Update consecutive attack counter
    if actual == 1:
        consecutive_attacks += 1
    else:
        consecutive_attacks = 0

    # IDS becomes aggressive only after 4 attacks in a row
    strategy = 'aggressive' if consecutive_attacks >= 4 else 'passive'

    prediction_prob = np.random.uniform(0.2, 0.5) if actual == 0 else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    if prediction == 1:
        print(f"[Packet {i+1}] 🚨 Intrusion Detected | Strategy: {strategy.upper()} | Prediction: {prediction_prob:.2f}")
        send_alert_email(i+1, prediction_prob, strategy, actual)

    # EGT update
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = (defender_strategy['aggressive'] * payoff_aggr +
                  defender_strategy['passive'] * payoff_pass)

    defender_strategy['aggressive'] += learning_rate * defender_strategy['aggressive'] * (payoff_aggr - avg_payoff)
    defender_strategy['passive'] += learning_rate * defender_strategy['passive'] * (payoff_pass - avg_payoff)

    total = defender_strategy['aggressive'] + defender_strategy['passive']
    defender_strategy['aggressive'] /= total
    defender_strategy['passive'] /= total

    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}")


[Packet 1] IDS State → Aggressive: 0.45, Passive: 0.55
[Packet 2] IDS State → Aggressive: 0.40, Passive: 0.60
[Packet 3] IDS State → Aggressive: 0.35, Passive: 0.65
[Packet 4] IDS State → Aggressive: 0.31, Passive: 0.69
[Packet 5] IDS State → Aggressive: 0.26, Passive: 0.74
[Packet 6] IDS State → Aggressive: 0.23, Passive: 0.77
[Packet 7] IDS State → Aggressive: 0.19, Passive: 0.81
[Packet 8] IDS State → Aggressive: 0.16, Passive: 0.84
[Packet 9] IDS State → Aggressive: 0.13, Passive: 0.87
[Packet 10] IDS State → Aggressive: 0.11, Passive: 0.89
[Packet 11] IDS State → Aggressive: 0.09, Passive: 0.91
[Packet 12] IDS State → Aggressive: 0.07, Passive: 0.93
[Packet 13] IDS State → Aggressive: 0.06, Passive: 0.94
[Packet 14] IDS State → Aggressive: 0.05, Passive: 0.95
[Packet 15] IDS State → Aggressive: 0.04, Passive: 0.96
[Packet 16] IDS State → Aggressive: 0.03, Passive: 0.97
[Packet 17] IDS State → Aggressive: 0.03, Passive: 0.97
[Packet 18] IDS State → Aggressive: 0.02, Passive: 0.98
[

In [ ]:
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_alert_email(packet_no, prediction_prob, strategy, actual):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    IDS Strategy: {strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[26:51]
df[' Label'] = df[' Label'].str.strip()
df['is_attack'] = df[' Label'].apply(lambda x: 'attack' if x.upper() != 'BENIGN' else 'benign')
df['attack_binary'] = df['is_attack'].apply(lambda x: 1 if x == 'attack' else 0)

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1
consecutive_attacks = 0

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']

    # Update consecutive attack counter
    if actual == 1:
        consecutive_attacks += 1
    else:
        consecutive_attacks = 0

    # IDS becomes aggressive only after 4 attacks in a row
    strategy = 'aggressive' if consecutive_attacks >= 4 else 'passive'

    prediction_prob = np.random.uniform(0.2, 0.5) if actual == 0 else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    if prediction == 1:
        print(f"[Packet {i+1}] 🚨 Intrusion Detected | Strategy: {strategy.upper()} | Prediction: {prediction_prob:.2f}")
        send_alert_email(i+1, prediction_prob, strategy, actual)

    # EGT update
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = (defender_strategy['aggressive'] * payoff_aggr +
                  defender_strategy['passive'] * payoff_pass)

    defender_strategy['aggressive'] += learning_rate * defender_strategy['aggressive'] * (payoff_aggr - avg_payoff)
    defender_strategy['passive'] += learning_rate * defender_strategy['passive'] * (payoff_pass - avg_payoff)

    total = defender_strategy['aggressive'] + defender_strategy['passive']
    defender_strategy['aggressive'] /= total
    defender_strategy['passive'] /= total

    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}")


[Packet 27] IDS State → Aggressive: 0.45, Passive: 0.55
[Packet 28] IDS State → Aggressive: 0.40, Passive: 0.60
[Packet 29] IDS State → Aggressive: 0.35, Passive: 0.65
[Packet 30] IDS State → Aggressive: 0.31, Passive: 0.69
[Packet 31] IDS State → Aggressive: 0.26, Passive: 0.74
[Packet 32] IDS State → Aggressive: 0.23, Passive: 0.77
[Packet 33] IDS State → Aggressive: 0.19, Passive: 0.81
[Packet 34] IDS State → Aggressive: 0.16, Passive: 0.84
[Packet 35] IDS State → Aggressive: 0.13, Passive: 0.87
[Packet 36] IDS State → Aggressive: 0.11, Passive: 0.89
[Packet 37] IDS State → Aggressive: 0.09, Passive: 0.91
[Packet 38] IDS State → Aggressive: 0.07, Passive: 0.93
[Packet 39] IDS State → Aggressive: 0.06, Passive: 0.94
[Packet 40] IDS State → Aggressive: 0.05, Passive: 0.95
[Packet 41] IDS State → Aggressive: 0.04, Passive: 0.96
[Packet 42] IDS State → Aggressive: 0.03, Passive: 0.97
[Packet 43] IDS State → Aggressive: 0.03, Passive: 0.97
[Packet 44] IDS State → Aggressive: 0.02, Passiv

In [ ]:
# Code with higher probability
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # App Password (for Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_alert_email(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    Strategy Used: {strategy.upper()}
    Dominant Strategy: {dominant_strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[0:26]  # Range 26–50

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'
    dominant = strategy  # same here since it's based on max

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    print(f"[Packet {i+1}] Strategy: {strategy.upper()} | Dominant: {dominant.upper()} | "
          f"Prediction: {prediction_prob:.2f} | Threshold: {threshold:.2f} | Actual: {'ATTACK' if actual else 'BENIGN'}")

    if prediction == 1:
        send_alert_email(i+1, prediction_prob, strategy, actual, dominant)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}\n")


[Packet 1] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.73 | Threshold: 0.70 | Actual: ATTACK
📧 Email alert sent for Packet 1
[Packet 1] IDS State → Aggressive: 0.57, Passive: 0.42

[Packet 2] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.32 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 2
[Packet 2] IDS State → Aggressive: 0.53, Passive: 0.47

[Packet 3] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.44 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 3
[Packet 3] IDS State → Aggressive: 0.48, Passive: 0.52

[Packet 4] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.81 | Threshold: 0.70 | Actual: ATTACK
📧 Email alert sent for Packet 4
[Packet 4] IDS State → Aggressive: 0.55, Passive: 0.45

[Packet 5] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.39 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 5
[Packet 5] IDS State → Aggressive: 0.50, Passive: 0.50

[Packet 6] Strategy: AGGRES

In [ ]:
# Code with higher probability
import pandas as pd
import numpy as np
import smtplib
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # App Password (for Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

def send_alert_email(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    Strategy Used: {strategy.upper()}
    Dominant Strategy: {dominant_strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[26:51]  # Range 26–50

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'
    dominant = strategy  # same here since it's based on max

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    print(f"[Packet {i+1}] Strategy: {strategy.upper()} | Dominant: {dominant.upper()} | "
          f"Prediction: {prediction_prob:.2f} | Threshold: {threshold:.2f} | Actual: {'ATTACK' if actual else 'BENIGN'}")

    if prediction == 1:
        send_alert_email(i+1, prediction_prob, strategy, actual, dominant)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}\n")


[Packet 27] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.80 | Threshold: 0.70 | Actual: ATTACK
📧 Email alert sent for Packet 27
[Packet 27] IDS State → Aggressive: 0.57, Passive: 0.42

[Packet 28] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.36 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 28
[Packet 28] IDS State → Aggressive: 0.53, Passive: 0.47

[Packet 29] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.66 | Threshold: 0.30 | Actual: ATTACK
📧 Email alert sent for Packet 29
[Packet 29] IDS State → Aggressive: 0.60, Passive: 0.40

[Packet 30] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.66 | Threshold: 0.30 | Actual: ATTACK
📧 Email alert sent for Packet 30
[Packet 30] IDS State → Aggressive: 0.67, Passive: 0.33

[Packet 31] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.64 | Threshold: 0.30 | Actual: ATTACK
📧 Email alert sent for Packet 31
[Packet 31] IDS State → Aggressive: 0.74, Passive: 0.26

[Packe

In [ ]:
import pandas as pd
import numpy as np
import smtplib
import requests
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # App Password (for Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

# -------- TELEGRAM CONFIG --------
TELEGRAM_BOT_TOKEN = "------------------------------------------"  # Replace with bot token
TELEGRAM_CHAT_ID = "**********"  # Replace with chat ID

def send_alert_email(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    Strategy Used: {strategy.upper()}
    Dominant Strategy: {dominant_strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

def send_telegram_alert(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    telegram_message = f"""
🚨 Intrusion Alert - Packet {packet_no}

Prediction Probability: {prediction_prob:.2f}
Strategy Used: {strategy.upper()}
Dominant Strategy: {dominant_strategy.upper()}
Ground Truth: {'ATTACK' if actual else 'BENIGN'}
"""

    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    params = {"chat_id": TELEGRAM_CHAT_ID, "text": telegram_message}

    try:
        response = requests.post(url, params=params)
        if response.status_code == 200:
            print(f"📨 Telegram alert sent for Packet {packet_no}")
        else:
            print(f"❌ Telegram failed for Packet {packet_no}: {response.text}")
    except Exception as e:
        print(f"❌ Telegram error for Packet {packet_no}: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[26:51]  # Range 26–50

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'
    dominant = strategy  # same here since it's based on max

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    print(f"[Packet {i+1}] Strategy: {strategy.upper()} | Dominant: {dominant.upper()} | "
          f"Prediction: {prediction_prob:.2f} | Threshold: {threshold:.2f} | Actual: {'ATTACK' if actual else 'BENIGN'}")

    if prediction == 1:
        send_alert_email(i+1, prediction_prob, strategy, actual, dominant)
        send_telegram_alert(i+1, prediction_prob, strategy, actual, dominant)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}\n")


[Packet 27] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.90 | Threshold: 0.70 | Actual: ATTACK
📧 Email alert sent for Packet 27
❌ Telegram failed for Packet 27: {"ok":false,"error_code":400,"description":"Bad Request: chat not found"}
[Packet 27] IDS State → Aggressive: 0.57, Passive: 0.42

[Packet 28] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.48 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 28
❌ Telegram failed for Packet 28: {"ok":false,"error_code":400,"description":"Bad Request: chat not found"}
[Packet 28] IDS State → Aggressive: 0.53, Passive: 0.47

[Packet 29] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.57 | Threshold: 0.30 | Actual: ATTACK
📧 Email alert sent for Packet 29
❌ Telegram failed for Packet 29: {"ok":false,"error_code":400,"description":"Bad Request: chat not found"}
[Packet 29] IDS State → Aggressive: 0.60, Passive: 0.40

[Packet 30] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.53 | Threshol

In [ ]:
import pandas as pd
import numpy as np
import smtplib
import requests
from email.mime.text import MIMEText

# -------- EMAIL CONFIG --------
EMAIL_SENDER = 'aditya27projects@gmail.com'
EMAIL_PASSWORD = '**** **** **** ****'  # App Password (for Gmail)
EMAIL_RECEIVER = 'aditya27kankarwal@gmail.com'

# -------- TELEGRAM CONFIG --------
TELEGRAM_BOT_TOKEN = "---------------------------------------"  # Replace with bot token
TELEGRAM_CHAT_ID = "**********"  # Replace with chat ID

def send_alert_email(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    subject = f"🚨 Intrusion Alert - Packet {packet_no}"
    body = f"""
    Intrusion Detected on Packet #{packet_no}

    Prediction Probability: {prediction_prob:.2f}
    Strategy Used: {strategy.upper()}
    Dominant Strategy: {dominant_strategy.upper()}
    Ground Truth: {'ATTACK' if actual else 'BENIGN'}
    """
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email alert sent for Packet {packet_no}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

def send_telegram_alert(packet_no, prediction_prob, strategy, actual, dominant_strategy):
    telegram_message = f"""
🚨 Intrusion Alert - Packet {packet_no}

Prediction Probability: {prediction_prob:.2f}
Strategy Used: {strategy.upper()}
Dominant Strategy: {dominant_strategy.upper()}
Ground Truth: {'ATTACK' if actual else 'BENIGN'}
"""

    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    params = {"chat_id": TELEGRAM_CHAT_ID, "text": telegram_message}

    try:
        response = requests.post(url, params=params)
        if response.status_code == 200:
            print(f"📨 Telegram alert sent for Packet {packet_no}")
        else:
            print(f"❌ Telegram failed for Packet {packet_no}: {response.text}")
    except Exception as e:
        print(f"❌ Telegram error for Packet {packet_no}: {e}")

# -------- LOAD DATA --------
df = pd.read_csv("/content/small_intrusion_dataset.csv").iloc[0:25]  # Range 0-25

# -------- EGT CONFIG --------
payoff_matrix = {
    ('aggressive', 'attack'): (3, -1),
    ('aggressive', 'benign'): (-1, 0),
    ('passive', 'attack'): (0, 3),
    ('passive', 'benign'): (1, 1),
}

defender_strategy = {'aggressive': 0.5, 'passive': 0.5}
learning_rate = 0.1

def update_strategy(strategy, actual):
    payoff_aggr = payoff_matrix[('aggressive', 'attack' if actual else 'benign')][0]
    payoff_pass = payoff_matrix[('passive', 'attack' if actual else 'benign')][0]
    avg_payoff = strategy['aggressive'] * payoff_aggr + strategy['passive'] * payoff_pass

    strategy['aggressive'] += learning_rate * strategy['aggressive'] * (payoff_aggr - avg_payoff)
    strategy['passive'] += learning_rate * strategy['passive'] * (payoff_pass - avg_payoff)

    total = strategy['aggressive'] + strategy['passive']
    strategy['aggressive'] /= total
    strategy['passive'] /= total

    return strategy

# -------- SIMULATION --------
for i, row in df.iterrows():
    actual = row['attack_binary']
    strategy = 'aggressive' if defender_strategy['aggressive'] > defender_strategy['passive'] else 'passive'
    dominant = strategy  # same here since it's based on max

    prediction_prob = np.random.uniform(0.2, 0.5) if not actual else np.random.uniform(0.5, 0.9)
    threshold = 0.3 if strategy == 'aggressive' else 0.7
    prediction = 1 if prediction_prob > threshold else 0

    print(f"[Packet {i+1}] Strategy: {strategy.upper()} | Dominant: {dominant.upper()} | "
          f"Prediction: {prediction_prob:.2f} | Threshold: {threshold:.2f} | Actual: {'ATTACK' if actual else 'BENIGN'}")

    if prediction == 1:
        send_alert_email(i+1, prediction_prob, strategy, actual, dominant)
        send_telegram_alert(i+1, prediction_prob, strategy, actual, dominant)

    defender_strategy = update_strategy(defender_strategy, actual)
    print(f"[Packet {i+1}] IDS State → Aggressive: {defender_strategy['aggressive']:.2f}, Passive: {defender_strategy['passive']:.2f}\n")


[Packet 1] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.52 | Threshold: 0.70 | Actual: ATTACK
[Packet 1] IDS State → Aggressive: 0.57, Passive: 0.42

[Packet 2] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.47 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 2
📨 Telegram alert sent for Packet 2
[Packet 2] IDS State → Aggressive: 0.53, Passive: 0.47

[Packet 3] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.23 | Threshold: 0.30 | Actual: BENIGN
[Packet 3] IDS State → Aggressive: 0.48, Passive: 0.52

[Packet 4] Strategy: PASSIVE | Dominant: PASSIVE | Prediction: 0.52 | Threshold: 0.70 | Actual: ATTACK
[Packet 4] IDS State → Aggressive: 0.55, Passive: 0.45

[Packet 5] Strategy: AGGRESSIVE | Dominant: AGGRESSIVE | Prediction: 0.41 | Threshold: 0.30 | Actual: BENIGN
📧 Email alert sent for Packet 5
📨 Telegram alert sent for Packet 5
[Packet 5] IDS State → Aggressive: 0.50, Passive: 0.50

[Packet 6] Strategy: AGGRESSIVE | Dominant: AGGRESSIV